In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.optimize import nnls

train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/spmm-latency-traintest/train-test-csv/nonsquare-train-1035-from-spmm-contain-todense-over-3s-1293.csv')
test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/spmm-latency-traintest/train-test-csv/nonsquare-test-258-from-spmm-contain-todense-over-3s-1293.csv')

In [2]:
# Train
X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc']] 
y_train = train['sp_smdm']

# Test
X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc']] 
y_test = test['sp_smdm']

In [3]:
def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [4]:
# NNLS(Non-Negative Least Squares) 회귀모델 정의 및 훈련데이터로 학습
# (모든 가중치가 0 또는 양수값을 가진다.)
nnls_coef, nnls_intercept =nnls(X_train.to_numpy(), y_train.to_numpy(), maxiter=10000)

In [5]:
# NNLS를 통해 나온 각 weight 추출
C1=np.array(pd.DataFrame(nnls_coef).iloc[0])
C2=np.array(pd.DataFrame(nnls_coef).iloc[1])
C3=np.array(pd.DataFrame(nnls_coef).iloc[2])
C4=np.array(pd.DataFrame(nnls_coef).iloc[3])
C5=np.array(pd.DataFrame(nnls_coef).iloc[4])
C6=np.array(pd.DataFrame(nnls_coef).iloc[5])
C7=np.array(pd.DataFrame(nnls_coef).iloc[6])
C8=np.array(pd.DataFrame(nnls_coef).iloc[7])
C9=np.array(pd.DataFrame(nnls_coef).iloc[8])
C10=np.array(pd.DataFrame(nnls_coef).iloc[9])

In [6]:
# 테스트 데이터 예측값 도출
y_pred = C1*X_test['lr']+C2*X_test['lc']+C3*X_test['rc']+C4*X_test['ld']+C5*X_test['rd']+C6*X_test['lnnz']+C7*X_test['rnnz']+C8*X_test['lr*lc']+C9*X_test['lc*rc']+C10*X_test['lr*rc']

print("테스트데이터 예측 mape : {}\n".format(mean_absolute_percentage_error(y_test,y_pred)))

테스트데이터 예측 mape : 252.21189098144401



In [7]:
mape_list = {}

# 예측값, 실제값을 확인하며 mape 계산 후 mape_list에 삽입 
for idx,value in enumerate(y_test):
    mape_temp = {}
    predicate = int(y_pred[idx])
    mape = abs((value - predicate) / value) * 100
    mape_temp['pred'] = predicate
    mape_temp['real'] = value
    mape_temp['mape'] = mape
    mape_list[idx] = mape_temp
mape_list_sort = sorted(mape_list.values(), key=lambda x:(x['mape']), reverse=True)
mape_list_sort  

[{'pred': 145115, 'real': 5183, 'mape': 2699.8263553926295},
 {'pred': 69319, 'real': 4916, 'mape': 1310.0691619202603},
 {'pred': 71919, 'real': 5549, 'mape': 1196.0713642097676},
 {'pred': 327330, 'real': 25478, 'mape': 1184.755475312034},
 {'pred': 40259, 'real': 3692, 'mape': 990.438786565547},
 {'pred': 30297, 'real': 3034, 'mape': 898.5827290705339},
 {'pred': 61867, 'real': 6321, 'mape': 878.7533618098403},
 {'pred': 55396, 'real': 5734, 'mape': 866.0969654691314},
 {'pred': 56239, 'real': 5854, 'mape': 860.6935428766656},
 {'pred': 45072, 'real': 5139, 'mape': 777.0577933450087},
 {'pred': 176974, 'real': 20290, 'mape': 772.2227698373583},
 {'pred': 58914, 'real': 7034, 'mape': 737.560420813193},
 {'pred': 58032, 'real': 6993, 'mape': 729.8584298584299},
 {'pred': 194356, 'real': 23694, 'mape': 720.275175149827},
 {'pred': 137134, 'real': 16860, 'mape': 713.3689205219455},
 {'pred': 25791, 'real': 3253, 'mape': 692.8373808791885},
 {'pred': 30032, 'real': 3975, 'mape': 655.5220

In [8]:
# 모델의 weight(가중치)와 bias(절편) 출력
print(nnls_coef)
print(nnls_intercept)

[0.00000000e+00 0.00000000e+00 0.00000000e+00 2.34281879e+06
 0.00000000e+00 1.49981676e-02 0.00000000e+00 0.00000000e+00
 1.03161949e-04 4.94171032e-05]
5852070.642346318


In [9]:
# nnls metric
print("R^2 : ", r2_score(y_test, y_pred))
print("\nMAPE : ", mean_absolute_percentage_error(y_test,y_pred))

R^2 :  0.6139435848995862

MAPE :  252.21189098144401
